In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from lifelines import CoxPHFitter
import statistics
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

%run functions.py


# load dataset
df_new = pd.read_csv('dataset_with_all_vars_converted_to_numerical_values.csv')
# determine the outcome label 
col_y='sab'

# do the needed preparation
## Center the “number of pregnancies” variable at the median for the gravid cohort
median_numpregs = df_new['b_numpregs'].median()
temp_numpregs = df_new.loc[(df_new['b_everpregnant']==1), 'b_numpregs']
df_new.loc[(df_new['b_everpregnant']==1), 'b_numpregs'] = temp_numpregs - median_numpregs

# Statistical Feature Selection (SFS) 
# step 1
# drop variables wit std of zero
df_std = df_new.std()
drop_cols = df_std[df_std<0.001].index.values
df_new = df_drop(df_new, drop_cols)

# compute the variables statistics and save it in dataframe named "result"
y = df_new[col_y].astype(int)
result = stat_test(df_new, y)

# step 2
# Compute pairwise correlation of variables with each other and determine pairs with corr > 0.9
# Compute the correlation of highly correlated pairs with outcome as well as their p-value
# We remove one variable among highly correlated pairs
selected_col_to_remove, df_highcorr = highcorr_stats(df_new, col_y, thres=0.9)
df_new = df_drop(df_new, selected_col_to_remove)

# step 3
# We test the association between each variable and the outcome 
# Chi2 test is used for non-continuous variables, and KS test is used for continuous variables
# We remove variables that are not independently associated with the outcome based on p-value > 0.05
drop_cols = result.loc[result['p-value']>0.05,'Variable'].values
df_new = df_drop(df_new, drop_cols)


# SURVIVAL MODEL DEVELOPMENT 

# define variables we use in the model definition
# in the format that cox model needs
df_new['col_time'] = (df_new['censorweek'] - df_new['startweek']).astype(int)
col_time = 'col_time'
col_strata = 'startweek'
# remove extra vars 
df_new = df_drop(df_new, ['censorweek'])

# define X as the model's input
X = df_new.drop([col_y, col_time, col_strata], axis=1)
# store features' names for final report
name_cols = df_new.drop([col_y, col_time, col_strata], axis=1).columns.values
# Standardization
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
# Save the dataset with standardized features 
df = pd.DataFrame(X, columns=name_cols)
df[col_y] = df_new[col_y]
df[col_time] = df_new[col_time]
df[col_strata] = df_new[col_strata]
y = df[col_y].astype(int)

# Split dataset to training set and testing set with test size of 0.2
cph_train, cph_test = train_test_split(df, test_size=0.2, random_state=2020, stratify=y) 

# FULL MODEL
# Train Cox model usig all variables
cph = CoxPHFitter(penalizer=0.01)
df_cph_train=pd.DataFrame(cph_train, columns=df.columns.tolist())
cph.fit(df=df_cph_train, duration_col=col_time, event_col=col_y, strata=col_strata)

# Report predictor variables' coefficients and hazard ratios based on CoxPH model
df_coef_cox = cph.summary[['coef', 'exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%']]
df_coef_cox['Variable'] = df_coef_cox.index
df_coef_cox['coef_abs'] = df_coef_cox['coef'].abs()
df_coef_cox = pd.merge(df_coef_cox,result[['Variable','p-value']], how='left', on=['Variable'])
LRresult_ = pd.merge(explain_Variables[['Variable','Label']],df_coef_cox.sort_values(by=['coef_abs'],ascending=False), how='right', on=['Variable'])
LRresult_=LRresult_.drop(['coef_abs'], axis=1)
print(LRresult_)

# Run 5 times with 5  different random spliting of dataset
# Report mean and std of models' concordance index on the testing set, over the 5 runs
test_score = []
train_score = []
my_seeds=range(2020, 2025)
for seed in my_seeds:  
    cph_train, cph_test = train_test_split(df, test_size=0.2, random_state=seed, stratify=y) 
    df_cph_train = pd.DataFrame(X_train_y, columns=df.columns.tolist())
    cph = CoxPHFitter(penalizer=0.01)
    cph.fit(df=df_cph_train, duration_col=col_time, event_col=col_y, strata=col_strata)
    train_score.append(cph.score(cph_train, scoring_method='concordance_index'))  
    test_score.append(cph.score(cph_test, scoring_method='concordance_index'))
print(f"The mean (std) of Concordance Index of Full models, over the 5 runs, on training set is: {statistics.mean(train_score)} ({np.std(train_score)})")
print(f"The mean (std) of Concordance Index of Full models, over the 5 runs, on testing set is: {statistics.mean(test_score)} ({np.std(test_score)})")


# SPARSE MODEL
# Univariate Feature Selection 
# Compute Concordance Index for each variable based on training set and select top k variables 
y = df_cph_train[[col_y, col_time, col_strata]]
X = df_cph_train.drop([col_y, col_time, col_strata], axis=1)
n_features = X.shape[1]
scores = np.empty(n_features)
m = CoxPHFitter(penalizer=0.01)
for j in range(n_features):
    Xj = X.iloc[:, j:j+1]
    Xj = pd.merge(Xj, y,  how='right', left_index=True, right_index=True)
    m.fit(Xj, duration_col=col_time, event_col=col_y, strata=col_strata)
    scores[j] = m.score(Xj, scoring_method='concordance_index')
# Store variables and their associated concordance index 
df_predictors = pd.Series(scores, index=X.columns).sort_values(ascending=False)
k=10 # number of variables we want to use in model 
top_predictors = df_predictors[0:k].index.tolist()
# store the dataset of selected features in df_top
df_top = df[top_predictors + [col_time, col_y, col_strata]]

# Train Cox model usig selected variables
cph = CoxPHFitter(penalizer=0.01)
cph.fit(df=df_cph_train[top_predictors+ [col_time, col_y, col_strata]], duration_col=col_time, event_col=col_y, strata=col_strata)

# Report predictor variables' coefficients and hazard ratios based on CoxPH model
df_coef_cox = cph.summary[['coef', 'exp(coef)', 'exp(coef) lower 95%', 'exp(coef) upper 95%']]
df_coef_cox['Variable'] = df_coef_cox.index
df_coef_cox['coef_abs'] = df_coef_cox['coef'].abs()
df_coef_cox = pd.merge(df_coef_cox,result[['Variable','p-value']], how='left', on=['Variable'])
LRresult_ = pd.merge(explain_Variables[['Variable','Label']],df_coef_cox.sort_values(by=['coef_abs'],ascending=False), how='right', on=['Variable'])
LRresult_=LRresult_.drop(['coef_abs'], axis=1)
print(LRresult_)

# Run 5 times with 5  different random spliting of dataset
# Report mean and std of models' concordance index on the testing set, over the 5 runs
test_score = []
train_score = []
for seed in my_seeds:  
    cph_train, cph_test = train_test_split(df_top, test_size=0.2, random_state=seed, stratify=y) 
    df_cph_train = pd.DataFrame(X_train_y, columns=df.columns.tolist())
    cph = CoxPHFitter(penalizer=0.01)
    cph.fit(df=df_cph_train, duration_col=col_time, event_col=col_y, strata=col_strata)
    train_score.append(cph.score(cph_train, scoring_method='concordance_index'))  
    test_score.append(cph.score(cph_test, scoring_method='concordance_index'))    
print(f"The mean (std) of Concordance Index of Sparse models, over the 5 runs, on training set is: {statistics.mean(train_score)} ({np.std(train_score)})")
print(f"The mean (std) of Concordance Index of Sparse models, over the 5 runs, on testing set is: {statistics.mean(test_score)} ({np.std(test_score)})")

